# Hurricane Emulator

In [1]:
# Convenient jupyter setup
%load_ext autoreload
%autoreload 2
%config IPCompleter.greedy=True

In [11]:
from typing import Tuple, Callable
import os
import numpy as np
import matplotlib.pyplot as plt
from adcircpy.outputs import Maxele
from sithom.plot import plot_defaults, label_subplots
from sithom.time import timeit
from sithom.place import Point
from src.conversions import si_ify, knots_to_ms
from src.constants import NEW_ORLEANS, DATA_PATH, NO_BBOX
from src.models.generation import ImpactSymmetricTC, Holland08

In [15]:
@np.vectorize
def indices_in_bbox(lon, lat):
    return (
        lon > NO_BBOX.lon[0]
        and lon < NO_BBOX.lon[1]
        and lat > NO_BBOX.lat[0]
        and lat < NO_BBOX.lat[1]
    )


@timeit
def func(rmax: float, folder: str, point: Point) -> float:
    output_direc = os.path.join(folder, "a" + "{:.3f}".format(rmax) + "_kat_rmax")
    ImpactSymmetricTC(
        point=point,
        output_direc=output_direc,
        symetric_model=Holland08(rmax=rmax),
    ).run_impact()
    path = os.path.join(output_direc, "maxele.63.nc")
    maxele = Maxele(path, crs="EPSG:4326")
    index_set = 27
    indices = indices_in_bbox(maxele.x, maxele.y)
    return maxele.values[indices][index_set]


@timeit
def function_sweep() -> Tuple[np.ndarray, np.ndarray]:
    rmax_min = 1e3
    rmax_max = 1e5
    rmaxes = np.linspace(rmax_min, rmax_max, num=1)

    lon_diff = 0.6
    point = Point(NEW_ORLEANS.lon + lon_diff, NEW_ORLEANS.lat)
    folder = os.path.join(DATA_PATH, "kat_emulation3")

    if not os.path.exists(folder):
        os.mkdir(folder)

    height_list = []

    for i in range(len(rmaxes)):
        height_list.append(func(rmaxes[i], folder, point))

    return rmaxes, np.array(height_list)


print(function_sweep())

'prepare_run'  12.23466 s 

'prepare_run'  42.90725 s 

'prepare_run'  11.74334 s 

'create_inputs'  01 min 06 s 

'run_adcirc'  04 min 54 s 

'func'  06 min 01 s 

'function_sweep'  06 min 01 s 

(array([1000.]), array([0.56463788]))


In [17]:
from typing import Callable


@timeit
def func_for_emukit() -> Callable:
    lon_diff = 0.6
    point = Point(NEW_ORLEANS.lon + lon_diff, NEW_ORLEANS.lat)
    folder = os.path.join(DATA_PATH, "kat_emulation")

    if not os.path.exists(folder):
        os.mkdir(folder)
        
    def _func(rmax):
        func(rmax, folder, point)
    
    return _func

In [19]:
f = func_for_emukit() 

'func_for_emukit'  0.00002 s 



In [ ]:
f(1e4)

'prepare_run'  11.76001 s 

